In [ ]:
import pandas as pd
import numpy as np

#выбор файлов
from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

def select_files(b):
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True) # List of selected files will be set button's file attribute.
    print(b.files) # Print the list of files selected.
    
    return b.files


In [ ]:
#выбрать входной файл
fileselect = Button(description="File select")
fileselect.on_click(select_files)

display(fileselect)

In [ ]:
for FILENAME in fileselect.files:
    #Путь и имя до входного файла
    #FILENAME = fileselect.files

    #Имя выходного файла
    FILENAME_OUT = f"{FILENAME[:-5]}_out.xlsx"

    #Прочитать данные из файла Excel
    DATA = pd.read_excel(f"{FILENAME}", dtype = str)

    #список вопросов
    #questions = DATA.iloc[0, 1:].values
    questions = DATA.columns[1:].values

    #переименовать столбцы (данные из 2 строки)
    #for i, name in  enumerate(DATA.columns):
    #    if i >= 6:
    #        old = name
    #        new = DATA.loc[0][i]
    #        DATA.rename(columns = {old : new}, inplace = True)

    #удалить первую строку
    #DATA.drop(index = 0, inplace = True)

    #удалить строки с пустой колонкой "Время финиша"
    #DATA_ED = DATA.dropna(subset = 'Время финиша')
    #DATA_ED = DATA_ED.replace({np.nan: 'Nan'})
    DATA_ED = DATA

    #Записать в Excel
    with pd.ExcelWriter(f"{FILENAME_OUT}", engine = 'xlsxwriter') as writer:

        # Get the xlsxwriter workbook and worksheet objects.
        workbook  = writer.book


        # Add some cell formats.
        format_percent = workbook.add_format({'num_format': '0%'})

        for i, vopros in enumerate(questions):
            vopros_num = vopros.split()[0]

            filter_col = [col for col in DATA_ED if col.startswith(vopros_num)]
            vopros_count = len(DATA_ED[filter_col].columns)


            if vopros_count != 1:
                #если вопрос - составной
                DATA_EDS = DATA_ED[filter_col]
                DATA_EDS = DATA_EDS.dropna(axis = 0, how='all', inplace = False)           
                DATA_EDS = DATA_EDS.replace({np.nan: 'Nan'})


                total_answers_count = DATA_EDS[vopros].count()

                srez = DATA_EDS[vopros].value_counts()
            else:
                total_answers_count = DATA_ED[vopros].count()

                srez = DATA_ED[vopros].value_counts()

            srez_pd = pd.DataFrame(srez)
            srez_pd = srez_pd.sort_index()
            srez_pd.loc['Всего ответов'] = total_answers_count

            srez_pd['Процент'] = (srez_pd[vopros] / total_answers_count)

            #srez_pd
            #srez_pd.to_excel(f"{i}.xlsx", index=True,) #save to xls file
            sheet_name = f"{i}_{vopros[:28]}"
            srez_pd.to_excel(writer, sheet_name = sheet_name)

            #Формат Excel файла
            worksheet = writer.sheets[sheet_name]

            (max_row, max_col) = srez_pd.shape


            # Set the format but not the column width.
            worksheet.set_column(2, 2, None, format_percent)

            # Make the columns wider for clarity.
            worksheet.set_column(0,  max_col, 40)

            # Set the autofilter.
            worksheet.autofilter(0, 0, max_row, max_col)

            print(f"{i} Запись в файл, вопрос {vopros}")